In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("pyspark_jupyter").master("spark://spark-master:7077").getOrCreate()
spark
data = [("Java", "20000"), ("Python", "100000"), ("Scala", "3000")]
df = spark.createDataFrame(data)
df.show()

df1 = spark.read.csv("/opt/spark/data/AAPL.csv")
df1.show()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/20 14:04:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                

+------+------+
|    _1|    _2|
+------+------+
|  Java| 20000|
|Python|100000|
| Scala|  3000|
+------+------+

+----------+------------------+------------------+------------------+------------------+-------------------+---------+
|       _c0|               _c1|               _c2|               _c3|               _c4|                _c5|      _c6|
+----------+------------------+------------------+------------------+------------------+-------------------+---------+
|      Date|              Open|              High|               Low|             Close|          Adj Close|   Volume|
|1980-12-12|0.5133928656578064|          0.515625|0.5133928656578064|0.5133928656578064|0.40678155422210693|117258400|
|1980-12-15|0.4888392984867096|0.4888392984867096|0.4866071343421936|0.4866071343421936|  0.385558158159256| 43971200|
|1980-12-16|          0.453125|          0.453125|0.4508928656578064|0.4508928656578064| 0.3572602868080139| 26432000|
|1980-12-17|0.4620535671710968|0.4642857015132904|0.46

In [ ]:
from pyspark.sql import SparkSession
import boto3

# Initialize Spark session with the necessary Hadoop AWS package
spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("PySpark with LocalStack S3") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localstack:4566") \
    .config("spark.hadoop.fs.s3a.access.key", "test") \
    .config("spark.hadoop.fs.s3a.secret.key", "test") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.connection.maximum", "100") \
    .config("spark.hadoop.fs.s3a.connection.timeout", "60000") \
    .config("spark.hadoop.fs.s3a.connection.acquisition.timeout", "60000") \
    .config("spark.hadoop.fs.s3a.read.timeout", "120000") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.jars", "/usr/local/lib/python3.10/dist-packages/pyspark/jars/hadoop-aws-3.2.0.jar,/usr/local/lib/python3.10/dist-packages/pyspark/jars/aws-java-sdk-bundle-1.11.375.jar,/usr/local/lib/python3.10/dist-packages/pyspark/jars/hadoop-common-3.2.0.jar") \
    .getOrCreate()
    
spark

s3 = boto3.client('s3', endpoint_url="http://localstack:4566", 
                  aws_access_key_id="test", aws_secret_access_key="test", region_name="us-east-1")

# Create a bucket in LocalStack's S3
bucket_name = 'my-test-bucket'
s3.create_bucket(Bucket=bucket_name)

# Upload a file to S3
# s3.put_object(Bucket=bucket_name, Key="test_file.txt", Body="This is a test file.")

# # Read from the bucket using PySpark
# df = spark.read.text(f"s3a://{bucket_name}/test_file.txt")

s3.list_buckets()
s3.upload_file("/opt/spark/data/AAPL.csv", 'my-test-bucket', 'AAPL')

objects = s3.list_objects_v2(Bucket='my-test-bucket')

for obj in objects['Contents']:
    print(obj['Key'])
# Stop Spark session
#spark.stop()
csv = spark.read.text("s3a://my-test-bucket/AAPL.csv")
csv.show()

response = s3.get_object(Bucket='my-test-bucket', Key='AAPL.csv')

object_content = response['Body'].read().decode('utf-8')

print(object_content)

In [9]:
from pyspark.sql import SparkSession

# Replace with your actual S3 bucket and path
s3_path = "s3a://my-test-bucket/AAPL.csv"

# Configure SparkSession
spark = SparkSession.builder.appName("LocalS3Connection") \
    .master("spark://spark-master:7077") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.hadoop.fs.s3a.access.key", "test") \
    .config("spark.hadoop.fs.s3a.secret.key", "test") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.0,com.amazonaws:aws-java-sdk-bundle:1.11.375") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:4566") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.connection.maximum", "100") \
    .config("spark.hadoop.fs.s3a.connection.timeout", "60000") \
    .config("spark.hadoop.fs.s3a.connection.acquisition.timeout", "60000") \
    .config("spark.hadoop.fs.s3a.read.timeout", "120000") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .getOrCreate()


# Read data from S3
try:
    df = spark.read.csv(s3_path, header=True, inferSchema=True)
    df.show()
except Exception as e:
    print(f"Error reading from S3: {e}")


Error reading from S3: For input string: "30s"


25/09/19 20:27:37 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/09/19 20:27:37 WARN FileSystem: Failed to initialize filesystem s3a://my-test-bucket/AAPL.csv: java.lang.NumberFormatException: For input string: "30s"
25/09/19 20:27:37 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: s3a://my-test-bucket/AAPL.csv.
java.lang.NumberFormatException: For input string: "30s"
	at java.base/java.lang.NumberFormatException.forInputString(NumberFormatException.java:67)
	at java.base/java.lang.Integer.parseInt(Integer.java:668)
	at java.base/java.lang.Integer.parseInt(Integer.java:786)
	at org.apache.hadoop.conf.Configuration.getInt(Configuration.java:1554)
	at org.apache.hadoop.fs.s3a.S3AUtils.intOption(S3AUtils.java:914)
	at org.apache.hadoop.fs.s3a.S3AUtils.initConnectionSettings(S3AUtils.java:1169)
	at org.apache.hadoop.fs.s3a.S3AUtils.createAwsConf(S3AUti